The purpose of this notebook is to help in managing the transitions within the config file to simulate various scenarios. It provides a function that ensures the probabilities in the transitions are valid, and an example of how to change the example-config file to your needs. The example given simulates the scenario where there is a sharp increase in the frequency of the 'Error' page appearing.

In [15]:
import json
with open("./example-config.json") as f:
    c = json.load(f)

In [23]:
def validate_probabilities(transitions):
    '''
    Validates that the transitions dictionary satisfies the probability constraint, i.e:

    For any source state s, the sum of P(s->d) over all possible destination states d must be <= 1
    '''
    check = {}
    for line in transitions:
        # a source state can be uniquely identified by (page, auth, level)
        src = line['source']
        src = (src['page'], src['auth'], src['level'])
        if src not in check:
            check[src] = 0
        check[src] += line['p']
    for src in check:
        if check[src] > 1:
            return False
    return True

The following cells change the transitions in the config file to one where all pages will lead to an 'Error' page with error code 408 with a very high probability of 0.9. After encountering an error page, the user has a 50% chance of ending the current session.

In [17]:
from copy import deepcopy
new_c = deepcopy(c)
prev = c['transitions'][0]['source']
prev = (prev['page'], prev['auth'], prev['level'])
accumulate_prev = []
for i in range(len(c['transitions'])):
    current = c['transitions'][i]['source']
    current = (current['page'], current['auth'], current['level'])
    if current == prev:
        accumulate_prev.append(i)
    else:
        for j in accumulate_prev:
            if prev[0] == 'Error':
                if new_c['transitions'][j]['dest']['page'] == 'Error':
                    new_c['transitions'][j]['dest']['status'] = 408
                    new_c['transitions'][j]['p'] = 0.4
                else:
                    new_c['transitions'][j]['p'] = 0.1/(len(accumulate_prev))
            else:
                if new_c['transitions'][j]['dest']['page'] == 'Error':
                    new_c['transitions'][j]['dest']['status'] = 408
                    new_c['transitions'][j]['p'] = 0.9
                else:
                    new_c['transitions'][j]['p'] = 0.1/(len(accumulate_prev))
        prev = current

        accumulate_prev = [i]

In [18]:
for i in range(5):
    print(c['transitions'][i])

{'source': {'page': 'About', 'method': 'GET', 'status': 200, 'auth': 'Guest', 'level': 'free'}, 'dest': {'page': 'About', 'method': 'GET', 'status': 200, 'auth': 'Guest', 'level': 'free'}, 'p': 0.05}
{'source': {'page': 'About', 'method': 'GET', 'status': 200, 'auth': 'Guest', 'level': 'free'}, 'dest': {'page': 'Help', 'method': 'GET', 'status': 200, 'auth': 'Guest', 'level': 'free'}, 'p': 0.05}
{'source': {'page': 'About', 'method': 'GET', 'status': 200, 'auth': 'Guest', 'level': 'free'}, 'dest': {'page': 'Home', 'method': 'GET', 'status': 200, 'auth': 'Guest', 'level': 'free'}, 'p': 0.3}
{'source': {'page': 'About', 'method': 'GET', 'status': 200, 'auth': 'Guest', 'level': 'free'}, 'dest': {'page': 'Register', 'method': 'GET', 'status': 200, 'auth': 'Guest', 'level': 'free'}, 'p': 0.1}
{'source': {'page': 'About', 'method': 'GET', 'status': 200, 'auth': 'Guest', 'level': 'free'}, 'dest': {'page': 'Error', 'method': 'GET', 'status': 404, 'auth': 'Guest', 'level': 'free'}, 'p': 0.001}


In [ ]:
for i in range(5):
    print(new_c['transitions'][i])

{'source': {'page': 'About', 'method': 'GET', 'status': 200, 'auth': 'Guest', 'level': 'free'}, 'dest': {'page': 'About', 'method': 'GET', 'status': 200, 'auth': 'Guest', 'level': 'free'}, 'p': 0.02}
{'source': {'page': 'About', 'method': 'GET', 'status': 200, 'auth': 'Guest', 'level': 'free'}, 'dest': {'page': 'Help', 'method': 'GET', 'status': 200, 'auth': 'Guest', 'level': 'free'}, 'p': 0.02}
{'source': {'page': 'About', 'method': 'GET', 'status': 200, 'auth': 'Guest', 'level': 'free'}, 'dest': {'page': 'Home', 'method': 'GET', 'status': 200, 'auth': 'Guest', 'level': 'free'}, 'p': 0.02}
{'source': {'page': 'About', 'method': 'GET', 'status': 200, 'auth': 'Guest', 'level': 'free'}, 'dest': {'page': 'Register', 'method': 'GET', 'status': 200, 'auth': 'Guest', 'level': 'free'}, 'p': 0.02}
{'source': {'page': 'About', 'method': 'GET', 'status': 200, 'auth': 'Guest', 'level': 'free'}, 'dest': {'page': 'Error', 'method': 'GET', 'status': 408, 'auth': 'Guest', 'level': 'free'}, 'p': 0.9}


In [24]:
validate_probabilities(new_c['transitions'])

True

The below cell changes the probability for the which state a new session starts on. This is based on the relative weights between the different possible start states and does not need to fulfill any probability constraints.

In [ ]:
for k in new_c['new-session']:
    if k['page'] == 'Error':
        k['status'] = 408
        k['weight'] = 50 if k['auth'] == 'Guest' else 2500

In [ ]:
with open('outage-config.json', 'w') as f:
    f.write(json.dumps(new_c, indent = 2))